In [1]:
from dataclasses import dataclass, field
from functools import lru_cache
from shapely import LineString, MultiPolygon, Polygon, difference
from typing import NamedTuple

@lru_cache
def manhattan(a, b):
    return sum(abs(ai - bi) for ai, bi in zip(a, b))

class Point(NamedTuple):
    x: int
    y: int

@dataclass
class Sensor:
    coords: Point
    beacon: Point
    distance: int = field(init=False)
    
    def __post_init__(self):
        self.distance = manhattan(self.coords, self.beacon)
        
    @property
    def zone(self):
        return Polygon((
            (self.coords.x, self.coords.y - self.distance),
            (self.coords.x + self.distance, self.coords.y),
            (self.coords.x, self.coords.y + self.distance),
            (self.coords.x - self.distance, self.coords.y),
        ))

In [2]:
import math
import re

regex_sensor = re.compile(r"(-?\d+)")

sensors = []
with open("Day15.txt") as file:
    for line in file:
        sx, sy, bx, by = map(int, regex_sensor.findall(line))
        sensors.append(Sensor(Point(sx, sy), Point(bx, by)))

In [3]:
%%time
limit = 2_000_000
zones = [sensor.zone for sensor in sensors]
area = MultiPolygon(zones).buffer(0)
xs, ys = area.exterior.coords.xy
line = LineString(((min(xs), limit), (max(xs), limit)))
int(area.intersection(line).length)

CPU times: user 5.94 ms, sys: 1.57 ms, total: 7.51 ms
Wall time: 6.31 ms


5144286

In [4]:
%%time
from functools import reduce
from shapely import Polygon, difference

limit = 4_000_000
area = Polygon(((0, 0), (0, limit), (limit, limit), (limit, 0)))
beacon = reduce(difference, zones, area).centroid
int(beacon.x * limit + beacon.y)

CPU times: user 6.68 ms, sys: 306 µs, total: 6.99 ms
Wall time: 5.5 ms


10229191267339